In [1]:
import os
%pwd

'g:\\End-to-End-Cancer-Cllassification-using-MLflow-and-DVC\\research'

In [2]:
os.chdir("../")

In [5]:
%pwd

'g:\\End-to-End-Cancer-Cllassification-using-MLflow-and-DVC'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sudipta.mahato.ece25/End-to-End-Cancer-Cllassification-using-MLflow-and-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sudipta.mahato.ece25"
os.environ["MLFLOW_TRACKING_PASSWORD"]="15862defaa59e6ab32355f30be42d5c4287bad8c"

In [ ]:
# MLFLOW_TRACKING_URI=https://dagshub.com/sudipta.mahato.ece25/End-to-End-Cancer-Cllassification-using-MLflow-and-DVC.mlflow \
# MLFLOW_TRACKING_USERNAME=sudipta.mahato.ece25 \
# MLFLOW_TRACKING_PASSWORD=15862defaa59e6ab32355f30be42d5c4287bad8c \
# python script.py

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    image_size: list
    batch_size: int 


In [13]:
from cnnClassifier.constants import *;
from cnnClassifier.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self)->EvaluationConfig:
        evaluation_config=EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            all_params=self.params,
            mlflow_uri="https://dagshub.com/sudipta.mahato.ece25/End-to-End-Cancer-Cllassification-using-MLflow-and-DVC.mlflow",
            image_size=self.params.IMAGE_SIZE,
            batch_size=self.params.BATCH_SIZE
        )

        return evaluation_config

In [36]:
import mlflow.keras
import tensorflow as tf
from cnnClassifier.utils.common import save_json
from urllib.parse import urlparse
import mlflow

class Evaluation:
    def __init__(self,config=EvaluationConfig):
        self.config=config
    
    def valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs=dict(
            target_size=self.config.image_size[:-1],
            batch_size=self.config.batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_datagenerator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            shuffle=False,
            subset='validation',
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        model=tf.keras.models.load_model(path)
        return model
        
    def evaluate(self):
        self.model=self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score=self.model.evaluate(self.valid_datagenerator)
        self.save_score()

    def save_score(self):
        scores={
                "loss":self.score[0],
                "accuarcy":self.score[1]
            }
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                    "loss":self.score[0],
                    "accuracy":self.score[1]
                })
            if tracking_url_type_store!='file':
                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")
            


In [37]:
from cnnClassifier.exception import CustomException
import sys
try:
    config=ConfigurationManager()
    evaluation_config=config.get_evaluation_config()
    evaluation=Evaluation(config=evaluation_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
    raise CustomException(e,sys)


[2024-08-02 13:32:41,888: INFO : common :yaml file:config\config.yaml loaded succesfully]
[2024-08-02 13:32:41,901: INFO : common :yaml file:params.yaml loaded succesfully]
[2024-08-02 13:32:41,906: INFO : common :Created directory at artifacts]


Found 102 images belonging to 2 classes.
4/4 [==============================] - 72s 13s/step - loss: 0.2688 - accuracy: 0.8431
[2024-08-02 13:34:04,126: INFO : common :json file saves at:scores.json]


2024/08/02 13:34:08 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-02 13:34:40,509: WARNING : save :Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-08-02 13:34:48,344: INFO : builder_impl :Assets written to: C:\Users\User\AppData\Local\Temp\tmpuop_hoz_\model\data\model\assets]


2024/08/02 13:37:14 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\User\AppData\Local\Temp\tmpuop_hoz_\model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\User\anaconda3\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/08/02 13:37:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
